In [1]:
import boto3

# Connect to MinIO
s3 = boto3.client(
    's3',
    endpoint_url='http://localhost:9000',  # MinIO endpoint
    aws_access_key_id='admin',
    aws_secret_access_key='password',
)

# List all buckets
buckets = s3.list_buckets()
print("Buckets:")
for b in buckets['Buckets']:
    print(" ", b['Name'])

# List files in your ECG path
bucket_name = 'warehouse'
prefix = 'fs/ecg_avro/'

objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
print(f"\nFiles in {prefix}:")
if 'Contents' in objects:
    for obj in objects['Contents']:
        print(" ", obj['Key'])
else:
    print("No files found")

Buckets:
  warehouse

Files in fs/ecg_avro/:
  fs/ecg_avro/D_01/22521218084687_4466f6e3-3ff0-4a50-a5ed-6b5f630be8cb.avro
  fs/ecg_avro/D_01/22521220252272_7205c751-7424-4152-8e07-52c6eca2ad1c.avro
  fs/ecg_avro/D_01/22522221817561_0f1922c6-e3d5-49c5-a9a6-d9d1d02c63a0.avro
  fs/ecg_avro/D_01/22522221872457_ffd5f171-1523-4c23-ad05-4a8300b19789.avro
  fs/ecg_avro/D_01/22522222908202_15e01d0c-59b3-4b25-9236-4a6567d97e6d.avro
  fs/ecg_avro/D_01/22522223341979_8a5585b4-c7d9-4397-8de8-cf0665f626b8.avro
  fs/ecg_avro/D_01/22522262471193_b6ba3962-750f-43a5-bba2-7fb1e88e4092.avro
  fs/ecg_avro/D_01/22522263379944_672cefda-f2f0-40ad-a62c-d420423bdc71.avro
  fs/ecg_avro/D_01/22523277482069_418cb67f-8cc6-4703-885b-432db5729947.avro
  fs/ecg_avro/D_01/22523281061679_32f95622-da0f-4087-905c-680266fb07f8.avro
  fs/ecg_avro/D_01/22523287404642_f2a7df90-f04c-414a-bb5e-44e4d8f9e5de.avro
  fs/ecg_avro/D_01/22534276739803_895902f0-a736-45a5-bd4e-98d018ba2c38.avro


In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Delta ECG Job - First Avro")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "io.delta:delta-spark_2.13:4.0.0,"
        "org.apache.hadoop:hadoop-aws:3.4.1,"
        "org.apache.spark:spark-avro_2.13:3.4.1"
    )
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.sql.warehouse.dir", "s3a://warehouse/")
    .getOrCreate()
)

# Now read Avro
avro_path = "s3a://warehouse/fs/ecg_avro/D_01/*.avro"
df = spark.read.format("avro").load(avro_path)

df.show(5, truncate=False)
print("Total rows read:", df.count())


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/28 19:35:14 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/28 19:35:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c2b4161-518a-4fd1-a66b-89f828f4b1e9;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in centr

Py4JJavaError: An error occurred while calling o51.load.
: java.net.UnknownHostException: listObjects() on s3a://warehouse/fs/ecg_avro/D_01: software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.:    software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.: minio
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.Native